In [312]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC, SVR
from sklearn import datasets
from sklearn.metrics import confusion_matrix, accuracy_score
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader 
import copy

import xgboost as xgb
import lightgbm as lgbm
import catboost as cbt
from sklearn_rvm import EMRVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from datetime import datetime
from sklearn.svm import OneClassSVM
from sklearn.mixture import GaussianMixture
from sklearn.ensemble import IsolationForest 
from sklearn.neighbors import LocalOutlierFactor

# 🌠 Anomaly Detection for Supervised Task

- Anoamly Detection은 근본적인 Supervised Classification Task를 Anoamly Detection으로 변경한 Task에여대하여 Anomaly Detection이 가능할까?

# Load Dataset

In [313]:
# import statsmodels.api as sm
# from tabulate import tabulate

# ces11 = sm.datasets.get_rdataset('CES11', 'carData')

# ces11.data.sample(12)

In [463]:
# random_seed
# rand_seed = 1024 
rand_seed = 2048 
# rand_seed = 4096

# Select Dataset

dataset_name = 'diabetes'
# dataset_name = 'breast_cancer'
# dataset_name = 'digits'
# dataset_name = 'iris'

if dataset_name == 'diabetes':
    df = pd.read_csv('diabetes.csv')
    X = df.iloc[:,:-1].values   
    y = df.iloc[:,-1].values    
    THRESHOLD_FOR_AUTOENCODER = 0.07
    THRESHOLD_FOR_DENSITY=36
    NUM_1ST_HIDDEN = 6 
    NUM_2ND_HIDDEN = 3 
    class_anomaly = 1

elif dataset_name == 'breast_cancer':
    breast_cancer = datasets.load_breast_cancer()
    X = breast_cancer.data
    y = breast_cancer.target
    THRESHOLD_FOR_AUTOENCODER = 1.35
    THRESHOLD_FOR_DENSITY = 70
    NUM_1ST_HIDDEN = 32 
    NUM_2ND_HIDDEN = 8 
    class_anomaly = 1

elif dataset_name == 'digits':
    digits = datasets.load_digits()
    X = digits.data
    y = digits.target

    THRESHOLD_FOR_AUTOENCODER = 8.5
    THRESHOLD_FOR_DENSITY = 23
    NUM_1ST_HIDDEN = 32 
    NUM_2ND_HIDDEN = 8 

    class_anomaly = 5

elif dataset_name == 'iris':
    iris = datasets.load_iris()
    X = iris.data
    y = iris.target
else:
    pass

In [464]:
print(f'Median {np.median(y)}')
print(f'Min {np.min(y)}')
print(f'Max {np.max(y)}')

Median 0.0
Min 0
Max 1


# Split data by specific threshold
- To convert regression problem to classification problem (two class for anomaly detection)

In [465]:
print("X shape")
print(X.shape)
print("Y shape")
print(y.shape)

X shape
(768, 8)
Y shape
(768,)


In [466]:
# split the dataset
x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=rand_seed, train_size=0.8)
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, random_state=rand_seed, train_size=0.8)



In [467]:
y_train_a = y_train.copy()
y_valid_a = y_valid.copy()
y_test_a = y_test.copy()


y_train_a[y_train==class_anomaly] = -1
y_train_a[y_train!=class_anomaly] = 1

y_valid_a[y_valid==class_anomaly] = -1
y_valid_a[y_valid!=class_anomaly] = 1

y_test_a[y_test==class_anomaly] = -1
y_test_a[y_test!=class_anomaly] = 1

In [468]:
# scaling by minmax scaler
scaler = MinMaxScaler()

scaler = scaler.fit(x_train)

x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)
x_valid = scaler.transform(x_valid)

In [469]:
print('valid', len(y_valid_a[y_valid_a==-1]))
print('train', len(y_train_a[y_train_a ==-1]))
print('test', len(y_test_a[y_test_a ==-1]))

print('len train origin : ', len(x_train))
x_train_only = x_train[y_train_a==1]
print('len train origin with only true : ', len(x_train_only))

valid 49
train 163
test 56
len train origin :  491
len train origin with only true :  328


# SVM (Support Vector Classification)
- Classification 

In [470]:
param_grid = [
    {'kernel': ['linear'], 'C': [1.0, 2.0, 3.0, 10.]},
    {'kernel': ['rbf'], 'C': [1.0, 2.0, 3.0, 5.0, 10.],
    'gamma': [0.01, 0.03, 0.1, 0.3, 1.0, 3.0]},
]

elapsed_time_kernel_svm = []

svm_classifier = SVC(kernel='rbf')
# svm_classifier = svm_classifier.fit(x_train, y_train)

start_time = datetime.now()
grid_search = GridSearchCV(svm_classifier, param_grid, cv=7, scoring="neg_mean_squared_error", verbose=2)
best_svc_classifier = grid_search.fit(x_train, y_train_a)
elapsed_time_kernel_svm.append((datetime.now()-start_time).total_seconds())



# y_pred = svm_classifier.predict(x_test)
# start_time = datetime.now()
# y_pred = best_svm_classifier.predict(x_test)
# elapsed_time_kernel_svm.append((datetime.now()-start_time).total_seconds())
# acc_svm_kernel = accuracy_score(y_test, y_pred)

Fitting 7 folds for each of 34 candidates, totalling 238 fits
[CV] END ...............................C=1.0, kernel=linear; total time=   0.0s
[CV] END ...............................C=1.0, kernel=linear; total time=   0.0s
[CV] END ...............................C=1.0, kernel=linear; total time=   0.0s
[CV] END ...............................C=1.0, kernel=linear; total time=   0.0s
[CV] END ...............................C=1.0, kernel=linear; total time=   0.0s
[CV] END ...............................C=1.0, kernel=linear; total time=   0.0s
[CV] END ...............................C=1.0, kernel=linear; total time=   0.0s
[CV] END ...............................C=2.0, kernel=linear; total time=   0.0s
[CV] END ...............................C=2.0, kernel=linear; total time=   0.0s
[CV] END ...............................C=2.0, kernel=linear; total time=   0.0s
[CV] END ...............................C=2.0, kernel=linear; total time=   0.0s
[CV] END ...............................C=2.0, 

In [471]:

# y_pred = svm_classifier.predict(x_test)
start_time = datetime.now()
y_pred = best_svc_classifier.predict(x_test)
elapsed_time_kernel_svm.append((datetime.now()-start_time).total_seconds())
acc_svm_kernel = accuracy_score(y_test_a, y_pred)



In [472]:

print('Confusion Matrix\n', confusion_matrix(y_test_a, y_pred))
print('Best Prameters ', grid_search.best_params_)
print('Accuracy ', acc_svm_kernel)
print('Elapsed Time(train, test) ', elapsed_time_kernel_svm)

Confusion Matrix
 [[28 28]
 [10 88]]
Best Prameters  {'C': 1.0, 'gamma': 0.3, 'kernel': 'rbf'}
Accuracy  0.7532467532467533
Elapsed Time(train, test)  [1.04221, 0.003001]


# One-Class SVM - Linear & Kernel

In [475]:
param_grid = [
    {'kernel': ['linear'], 'nu': [0.1, 0.25, 0,5, 0.7]},
    {'kernel': ['rbf'], 'nu': [0.1, 0.25, 0,5, 0.7],
    'gamma': [0.01, 0.03, 0.1, 0.3, 1.0]},
]

elapsed_time_kernel_svm = []

svm_classifier = OneClassSVM(kernel='rbf')
# svm_classifier = svm_classifier.fit(x_train, y_train)

start_time = datetime.now()
grid_search = GridSearchCV(svm_classifier, param_grid, cv=7, scoring="neg_mean_squared_error", verbose=2)
best_svm_classifier = grid_search.fit(x_train_only)
elapsed_time_kernel_svm.append((datetime.now()-start_time).total_seconds())


start_time = datetime.now()
y_pred = best_svm_classifier.predict(x_test)
elapsed_time_kernel_svm.append((datetime.now()-start_time).total_seconds())
acc_svm_kernel = accuracy_score(y_test_a, y_pred)



Fitting 7 folds for each of 30 candidates, totalling 210 fits
[CV] END ..............................kernel=linear, nu=0.1; total time=   0.0s
[CV] END ..............................kernel=linear, nu=0.1; total time=   0.0s
[CV] END ..............................kernel=linear, nu=0.1; total time=   0.0s
[CV] END ..............................kernel=linear, nu=0.1; total time=   0.0s
[CV] END ..............................kernel=linear, nu=0.1; total time=   0.0s
[CV] END ..............................kernel=linear, nu=0.1; total time=   0.0s
[CV] END ..............................kernel=linear, nu=0.1; total time=   0.0s
[CV] END .............................kernel=linear, nu=0.25; total time=   0.0s
[CV] END .............................kernel=linear, nu=0.25; total time=   0.0s
[CV] END .............................kernel=linear, nu=0.25; total time=   0.0s
[CV] END .............................kernel=linear, nu=0.25; total time=   0.0s
[CV] END .............................kernel=li

c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
c:\venv_python_3.10\lib\site-packa

[CV] END ......................gamma=0.1, kernel=rbf, nu=0.7; total time=   0.0s
[CV] END ......................gamma=0.1, kernel=rbf, nu=0.7; total time=   0.0s
[CV] END ......................gamma=0.1, kernel=rbf, nu=0.7; total time=   0.0s
[CV] END ......................gamma=0.1, kernel=rbf, nu=0.7; total time=   0.0s
[CV] END ......................gamma=0.1, kernel=rbf, nu=0.7; total time=   0.0s
[CV] END ......................gamma=0.1, kernel=rbf, nu=0.7; total time=   0.0s
[CV] END ......................gamma=0.1, kernel=rbf, nu=0.7; total time=   0.0s
[CV] END ......................gamma=0.3, kernel=rbf, nu=0.1; total time=   0.0s
[CV] END ......................gamma=0.3, kernel=rbf, nu=0.1; total time=   0.0s
[CV] END ......................gamma=0.3, kernel=rbf, nu=0.1; total time=   0.0s
[CV] END ......................gamma=0.3, kernel=rbf, nu=0.1; total time=   0.0s
[CV] END ......................gamma=0.3, kernel=rbf, nu=0.1; total time=   0.0s
[CV] END ...................

c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
c:\venv_python_3.10\lib\site-packa

In [476]:
print('Confusion Matrix\n', confusion_matrix(y_test_a, y_pred))
print('Best Prameters ', grid_search.best_params_)
print('Accuracy ', acc_svm_kernel)
print('Elapsed Time(train, test) ', elapsed_time_kernel_svm)
# Isolation Forest 

Confusion Matrix
 [[ 1 55]
 [ 7 91]]
Best Prameters  {'kernel': 'linear', 'nu': 0.1}
Accuracy  0.5974025974025974
Elapsed Time(train, test)  [0.310986, 0.0]


# Isolation Forest

In [477]:
iforest_classifier = IsolationForest()

iforest_parameters ={'contamination' : [0.01, 0.04, 0.08, 0.12, 0.16] , 'n_estimators': [4, 8, 12,24,32, 64]}
#  'learning_rate' : [0.01,0.02,0.03],
#  'gamma': [0.5, 1, 2], 'random_state':[99]}
elapsed_time_iforest = []

iforest_parameters = {'n_estimators': list(range(100, 200, 50)), 
              'max_samples': list(range(100, 110, 5)), 
              'contamination': [0.1, 0.2], 
              'max_features': [5,15], 
              'bootstrap': [True, False], 
              }


start_time = datetime.now()
iforest_grid_search = GridSearchCV(iforest_classifier, iforest_parameters, cv=7, scoring="neg_mean_squared_error", verbose=2)
best_iforest_classifier = iforest_grid_search.fit(x_train_only)

# iforest_grid_search = GridSearchCV(iforest_classifier, xgb_parameters, cv=7, scoring="neg_mean_squared_error", verbose=2)
# iforest_classifier.fit(x_train_only)
elapsed_time_iforest.append((datetime.now()-start_time).total_seconds())

Fitting 7 folds for each of 32 candidates, totalling 224 fits
[CV] END bootstrap=True, contamination=0.1, max_features=5, max_samples=100, n_estimators=100; total time=   0.0s
[CV] END bootstrap=True, contamination=0.1, max_features=5, max_samples=100, n_estimators=100; total time=   0.0s


c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
c:\venv_python_3.10\lib\site-packa

[CV] END bootstrap=True, contamination=0.1, max_features=5, max_samples=100, n_estimators=100; total time=   0.0s
[CV] END bootstrap=True, contamination=0.1, max_features=5, max_samples=100, n_estimators=100; total time=   0.0s
[CV] END bootstrap=True, contamination=0.1, max_features=5, max_samples=100, n_estimators=100; total time=   0.0s


c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
c:\venv_python_3.10\lib\site-packa

[CV] END bootstrap=True, contamination=0.1, max_features=5, max_samples=100, n_estimators=100; total time=   0.0s
[CV] END bootstrap=True, contamination=0.1, max_features=5, max_samples=100, n_estimators=100; total time=   0.0s


...

c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(


[CV] END bootstrap=False, contamination=0.2, max_features=5, max_samples=105, n_estimators=150; total time=   0.1s
[CV] END bootstrap=False, contamination=0.2, max_features=5, max_samples=105, n_estimators=150; total time=   0.0s


c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(


[CV] END bootstrap=False, contamination=0.2, max_features=5, max_samples=105, n_estimators=150; total time=   0.0s
[CV] END bootstrap=False, contamination=0.2, max_features=5, max_samples=105, n_estimators=150; total time=   0.0s
[CV] END bootstrap=False, contamination=0.2, max_features=5, max_samples=105, n_estimators=150; total time=   0.0s
[CV] END bootstrap=False, contamination=0.2, max_features=15, max_samples=100, n_estimators=100; total time=   0.0s
[CV] END bootstrap=False, contamination=0.2, max_features=15, max_samples=100, n_estimators=100; total time=   0.0s
[CV] END bootstrap=False, contamination=0.2, max_features=15, max_samples=100, n_estimators=100; total time=   0.0s
[CV] END bootstrap=False, contamination=0.2, max_features=15, max_samples=100, n_estimators=100; total time=   0.0s
[CV] END bootstrap=False, contamination=0.2, max_features=15, max_samples=100, n_estimators=100; total time=   0.0s
[CV] END bootstrap=False, contamination=0.2, max_features=15, max_samples=1

c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
112 fits failed out of a total of 224.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
112 fits failed with the following error:
Traceback (most recent call last):

In [478]:
# y_pred = xgb_classifier.predict(x_test)
start_time = datetime.now()
y_pred_c = best_iforest_classifier.predict(x_test)
elapsed_time_iforest.append((datetime.now()-start_time).total_seconds())


acc_iforest = accuracy_score(y_test_a, y_pred_c)

print('Confusion Matrix\n', confusion_matrix(y_test_a, y_pred_c))
print("best parameters ", iforest_grid_search.best_params_)
print('Accuracy ', acc_iforest)
print('elapsed time ', elapsed_time_iforest)

Confusion Matrix
 [[21 35]
 [11 87]]
best parameters  {'bootstrap': True, 'contamination': 0.1, 'max_features': 5, 'max_samples': 100, 'n_estimators': 100}
Accuracy  0.7012987012987013
elapsed time  [13.509137, 0.023573]


# Autoencoder for Anomaly Detection

In [479]:
class TrainData(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)



class TestData(Dataset):
    
    def __init__(self, X_data):
        self.X_data = X_data
        
    def __getitem__(self, index):
        return self.X_data[index]
        
    def __len__ (self):
        return len(self.X_data)
    

train_data = TrainData(torch.FloatTensor(x_train_only), 
                       torch.FloatTensor(x_train_only))

test_data = TestData(torch.FloatTensor(x_test))
valid_data = TrainData(torch.FloatTensor(x_valid), torch.FloatTensor(x_valid))

train_loader = DataLoader(dataset=train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=1)
valid_loader = DataLoader(dataset=valid_data, batch_size=2)

In [480]:
EPOCHS = 600 
BATCH_SIZE = 128 
LEARNING_RATE = 0.001

NUM_INPUT = x_train.shape[1]
NUM_OUTPUT = x_train.shape[1] 
NUM_1ST_HIDDEN = 32 
NUM_2ND_HIDDEN = 8 
NUM_1ST_DROPOUT = 0.1
NUM_2ND_DROPOUT = 0.1
NUM_3RD_DROPOUT = 0.1

In [481]:
class BasicClassification(nn.Module):
    def __init__(self) -> None:
        super(BasicClassification, self).__init__()

        self.layer_1 = nn.Linear(NUM_INPUT, NUM_1ST_HIDDEN)
        self.layer_2 = nn.Linear(NUM_1ST_HIDDEN, NUM_2ND_HIDDEN)
        self.layer_3 = nn.Linear(NUM_2ND_HIDDEN, NUM_1ST_HIDDEN)
        self.layer_4 = nn.Linear(NUM_1ST_HIDDEN, NUM_INPUT)

        # self.actvation = nn.ReLU()
        self.actvation_1 = nn.SELU()
        self.actvation_2 = nn.SELU()
        self.actvation_3 = nn.SELU()

        # self.actvation_1 = nn.ReLU()
        # self.actvation_2 = nn.ReLU()
        # self.actvation_3 = nn.ReLU()

        self.dropout_1 = nn.Dropout(p=NUM_1ST_DROPOUT)
        self.dropout_2 = nn.Dropout(p=NUM_2ND_DROPOUT)
        self.dropout_3 = nn.Dropout(p=NUM_3RD_DROPOUT)
        self.batchnorm_1 = nn.BatchNorm1d(NUM_1ST_HIDDEN)
        self.batchnorm_2 = nn.BatchNorm1d(NUM_2ND_HIDDEN)
        self.batchnorm_3 = nn.BatchNorm1d(NUM_1ST_HIDDEN)
    
    def forward(self, inputs):
        x = self.actvation_1(self.layer_1(inputs))
        # x = self.batchnorm_1(x)
        # x = self.dropout_1(x)
        x = self.actvation_2(self.layer_2(x))
        # x = self.batchnorm_2(x)
        # x = self.dropout_2(x)

        x = self.actvation_3(self.layer_3(x))
        # x = self.batchnorm_3(x)
        # x = self.dropout_3(x)

        # x = self.actvation_4(self.layer_4(x))
        # x = self.batchnorm_3(x)
        # x = self.dropout_3(x)


        x = self.layer_4(x)

        return x


In [482]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [483]:
model = BasicClassification()
model.to(device)

print(model)


# criterion = nn.CrossEntropyLoss()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

BasicClassification(
  (layer_1): Linear(in_features=8, out_features=32, bias=True)
  (layer_2): Linear(in_features=32, out_features=8, bias=True)
  (layer_3): Linear(in_features=8, out_features=32, bias=True)
  (layer_4): Linear(in_features=32, out_features=8, bias=True)
  (actvation_1): SELU()
  (actvation_2): SELU()
  (actvation_3): SELU()
  (dropout_1): Dropout(p=0.1, inplace=False)
  (dropout_2): Dropout(p=0.1, inplace=False)
  (dropout_3): Dropout(p=0.1, inplace=False)
  (batchnorm_1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm_2): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm_3): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


In [484]:
best_model  = None
best_loss_on_valid = 999999999
num_train_data = len(train_loader)
num_eval_data = len(valid_loader)


elapsed_time_basic_ann = []

start_time = datetime.now()


for epoch in range(1, EPOCHS+1):
    epoch_loss = 0
    epoch_acc = 0

    eval_epoch_loss = 0
    eval_epoch_acc = 0

    model.train()
    for x_batch, y_batch in train_loader:
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()

        y_pred = model(x_batch)

        loss = criterion(y_pred, y_batch)
        # acc = calc_accuracy(y_pred, y_batch)

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        # epoch_acc += acc.item()
    
    if epoch % 10 == 0:
        
        with torch.no_grad():
            model.eval()
            for x, y in valid_loader:
                x = x.to(device)
                y = y.to(device)

                output = model(x)

                eval_loss = criterion(output, y)
                # eval_acc = calc_accuracy(output, y)

                eval_epoch_loss += eval_loss.item()
                # eval_epoch_acc += eval_acc.item()
        
        if best_loss_on_valid >= (eval_epoch_loss/num_eval_data):
            best_loss_on_valid = (eval_epoch_loss/num_eval_data)
            best_model = copy.deepcopy(model)
            print("Best Model is copied - Best Loss : ", best_loss_on_valid)
        


        print(f"Epoch {epoch+0:03}: : Loss: T_{epoch_loss/num_train_data:.3f} V_{eval_epoch_loss/num_eval_data:.3f} | Acc: T_{epoch_acc/num_train_data:.3f}) V_{eval_epoch_acc/num_eval_data:.3f}")


elapsed_time_basic_ann.append((datetime.now()-start_time).total_seconds())

Best Model is copied - Best Loss :  0.019195076378603137
Epoch 010: : Loss: T_0.017 V_0.019 | Acc: T_0.000) V_0.000
Best Model is copied - Best Loss :  0.009404742296019028
Epoch 020: : Loss: T_0.008 V_0.009 | Acc: T_0.000) V_0.000
Best Model is copied - Best Loss :  0.005782315793875305
Epoch 030: : Loss: T_0.005 V_0.006 | Acc: T_0.000) V_0.000
Best Model is copied - Best Loss :  0.003673583446558745
Epoch 040: : Loss: T_0.003 V_0.004 | Acc: T_0.000) V_0.000
Best Model is copied - Best Loss :  0.002645903509185319
Epoch 050: : Loss: T_0.002 V_0.003 | Acc: T_0.000) V_0.000
Best Model is copied - Best Loss :  0.0019827265345919577
Epoch 060: : Loss: T_0.002 V_0.002 | Acc: T_0.000) V_0.000
Best Model is copied - Best Loss :  0.0012462878315296444
Epoch 070: : Loss: T_0.001 V_0.001 | Acc: T_0.000) V_0.000
Best Model is copied - Best Loss :  0.000850345068395273
Epoch 080: : Loss: T_0.001 V_0.001 | Acc: T_0.000) V_0.000
Best Model is copied - Best Loss :  0.0005046726348096385
Epoch 090: :

In [485]:

# with torch.no_grad():
# model.eval()
best_model.eval()
data = torch.from_numpy(x_test).float().to(device)
# answer = torch.from_numpy(y_test_onehot).float().to(device)


# data = torch.from_numpy(x_train).float().to(device)
# answer = torch.from_numpy(y_train_onehot).float().to(device)


# data = torch.from_numpy(x_valid).float().to(device)
# answer = torch.from_numpy(y_valid_onehot).float().to(device)

start_time = datetime.now()
output = best_model(data)
# acc_basic_ann = calc_accuracy(output, answer)
elapsed_time_basic_ann.append((datetime.now()-start_time).total_seconds())

# print('Accuracy ', acc_basic_ann)
print('elapsed time ', elapsed_time_basic_ann)

elapsed time  [24.526371, 0.001055]


In [486]:

output_num = output.detach().cpu().numpy()



In [487]:
output_num[0]

array([ 0.1359435 ,  0.70735896,  0.68146825,  0.0065643 , -0.0011842 ,
        0.3891418 ,  0.03504434,  0.16217749], dtype=float32)

In [488]:
x_test[0]

array([0.13333333, 0.69849246, 0.68181818, 0.        , 0.        ,
       0.38152012, 0.03800171, 0.15686275])

In [489]:
result_reconstruct = abs(x_test - output_num).sum(axis=1)
print(len(result_reconstruct))
print(len(x_test))
print(result_reconstruct)

154
154
[0.03546888 0.0405087  0.02703935 0.06068416 0.01883283 0.04764835
 0.02617201 0.03209207 0.03119371 0.16038885 0.06047502 0.02544065
 0.1123743  0.33049339 0.27329982 0.04722601 0.44233158 0.03216962
 0.02596441 0.02689224 0.13051849 0.06732572 0.03774988 0.02823789
 0.06229838 0.07264621 0.05290809 0.0257998  0.04273094 0.03195111
 0.06033531 0.03851215 0.19442281 0.06096279 0.02418812 0.04310794
 0.02515596 0.24877368 0.03602096 0.05099623 0.0577322  0.03914136
 0.03655293 0.04004613 0.03542421 0.02637027 0.17724267 0.04996319
 0.02856521 0.02581766 0.05414408 0.03730743 0.03153666 0.06819111
 0.0237403  0.02461978 0.05390628 0.0284217  0.59679032 0.03317379
 0.05132814 0.06413137 0.02902826 0.02511179 0.0448418  0.19781726
 0.03728136 0.10348703 0.20941368 0.0644788  0.03138812 0.02094049
 0.04036439 0.03301287 0.05280438 0.09864354 0.04273773 0.04724109
 0.04024349 0.04698941 0.18633917 0.12915356 0.03045529 0.15275484
 0.03568011 0.04952545 0.01991763 0.04589357 0.0481104

In [589]:

result_class = result_reconstruct.copy()
result_class[result_reconstruct > THRESHOLD_FOR_AUTOENCODER] = -1
result_class[result_reconstruct <= THRESHOLD_FOR_AUTOENCODER] = 1

In [590]:
# result_class
acc_ae = accuracy_score(y_test_a, result_class)

print('Confusion Matrix\n', confusion_matrix(y_test_a, result_class))
print('Accuracy ', acc_ae)

Confusion Matrix
 [[18 38]
 [16 82]]
Accuracy  0.6493506493506493


In [187]:
len(output_num)

360

# Mixture of Gaussian

In [591]:
gmm_classifier = GaussianMixture()

# lgbm_classifier=lgbm_classifier.fit(x_train, y_train)
# y_pred = lgbm_classifier.predict(x_test)


gmm_parameters ={'n_components' : [1, 2, 3,4,5,6, 7] , 'max_iter': [int(1e2), int(1e3), int(1e6)]}
#  'learning_rate' : [0.01,0.02,0.03],
#  'gamma': [0.5, 1, 2], 'random_state':[99]}

elapsed_time_gmm= []

start_time = datetime.now()
gmm_grid_search = GridSearchCV(gmm_classifier, gmm_parameters, cv=7, scoring="neg_mean_squared_error", verbose=2)
best_gmm_classifier = gmm_grid_search.fit(x_train_only)
elapsed_time_gmm.append((datetime.now()-start_time).total_seconds())




# acc_lgbm = accuracy_score(y_test, y_pred)

# print('Confusion Matrix\n', confusion_matrix(y_test, y_pred))
# print("best parameters ", lgbm_grid_search.best_params_)
# print('Accuracy ', acc_lgbm)
# print('elapsed time ', elapsed_time_lgbm)


Fitting 7 folds for each of 21 candidates, totalling 147 fits
[CV] END .......................max_iter=100, n_components=1; total time=   0.0s
[CV] END .......................max_iter=100, n_components=1; total time=   0.0s
[CV] END .......................max_iter=100, n_components=1; total time=   0.0s
[CV] END .......................max_iter=100, n_components=1; total time=   0.0s
[CV] END .......................max_iter=100, n_components=1; total time=   0.0s
[CV] END .......................max_iter=100, n_components=1; total time=   0.0s
[CV] END .......................max_iter=100, n_components=1; total time=   0.0s
[CV] END .......................max_iter=100, n_components=2; total time=   0.0s
[CV] END .......................max_iter=100, n_components=2; total time=   0.0s
[CV] END .......................max_iter=100, n_components=2; total time=   0.0s
[CV] END .......................max_iter=100, n_components=2; total time=   0.0s
[CV] END .......................max_iter=100, n

c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
c:\venv_python_3.10\lib\site-packa

[CV] END .......................max_iter=100, n_components=3; total time=   0.0s
[CV] END .......................max_iter=100, n_components=3; total time=   0.0s
[CV] END .......................max_iter=100, n_components=3; total time=   0.0s
[CV] END .......................max_iter=100, n_components=3; total time=   0.0s
[CV] END .......................max_iter=100, n_components=3; total time=   0.0s
[CV] END .......................max_iter=100, n_components=3; total time=   0.0s
[CV] END .......................max_iter=100, n_components=4; total time=   0.0s
[CV] END .......................max_iter=100, n_components=4; total time=   0.0s
[CV] END .......................max_iter=100, n_components=4; total time=   0.0s


c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
c:\venv_python_3.10\lib\site-packa

[CV] END .......................max_iter=100, n_components=4; total time=   0.0s
[CV] END .......................max_iter=100, n_components=4; total time=   0.0s
[CV] END .......................max_iter=100, n_components=4; total time=   0.0s
[CV] END .......................max_iter=100, n_components=4; total time=   0.0s
[CV] END .......................max_iter=100, n_components=5; total time=   0.0s
[CV] END .......................max_iter=100, n_components=5; total time=   0.0s


...

c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
c:\venv_python_3.10\lib\site-packa

[CV] END ...................max_iter=1000000, n_components=5; total time=   0.0s
[CV] END ...................max_iter=1000000, n_components=5; total time=   0.0s
[CV] END ...................max_iter=1000000, n_components=6; total time=   0.0s
[CV] END ...................max_iter=1000000, n_components=6; total time=   0.0s
[CV] END ...................max_iter=1000000, n_components=6; total time=   0.0s
[CV] END ...................max_iter=1000000, n_components=6; total time=   0.0s


c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
c:\venv_python_3.10\lib\site-packa

[CV] END ...................max_iter=1000000, n_components=6; total time=   0.0s
[CV] END ...................max_iter=1000000, n_components=6; total time=   0.0s
[CV] END ...................max_iter=1000000, n_components=6; total time=   0.0s
[CV] END ...................max_iter=1000000, n_components=7; total time=   0.0s
[CV] END ...................max_iter=1000000, n_components=7; total time=   0.0s
[CV] END ...................max_iter=1000000, n_components=7; total time=   0.0s
[CV] END ...................max_iter=1000000, n_components=7; total time=   0.0s
[CV] END ...................max_iter=1000000, n_components=7; total time=   0.0s
[CV] END ...................max_iter=1000000, n_components=7; total time=   0.0s
[CV] END ...................max_iter=1000000, n_components=7; total time=   0.0s


c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
c:\venv_python_3.10\lib\site-packa

In [592]:
start_time = datetime.now()
y_pred_c = best_gmm_classifier.predict(x_test)
elapsed_time_gmm.append((datetime.now()-start_time).total_seconds())

In [593]:
y_pred_c

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=int64)

In [594]:
print(len(np.argwhere(y_test_a == -1)))

56


In [623]:
densities = best_gmm_classifier.score_samples(x_test)
density_threshold = np.percentile(densities, THRESHOLD_FOR_DENSITY)
anomalies = np.argwhere(densities < density_threshold)
print(len(anomalies))
# print(anomalies)

# print(density_threshold)
# print(len(anomalies))
# print(np.argwhere(y_test_c == -1))
real_anomaly = np.argwhere(y_test_a == -1)
# print(real_)

y_pred_anomalies = y_test_a.copy()
y_pred_anomalies[densities < density_threshold] = -1
y_pred_anomalies[densities >= density_threshold] = 1


acc_iforest = accuracy_score(y_test_a, y_pred_anomalies)

print('Confusion Matrix\n', confusion_matrix(y_test_a, y_pred_anomalies))
print("best parameters ", best_gmm_classifier.best_params_)
print('Accuracy ', acc_iforest)
print('elapsed time ', elapsed_time_gmm)

# accuracy_score(y_test_c, y_pred_anomalies)

56
Confusion Matrix
 [[32 24]
 [24 74]]
best parameters  {'max_iter': 100, 'n_components': 1}
Accuracy  0.6883116883116883
elapsed time  [3.995679, 0.001]


In [597]:
# y_pred_anomalies

In [74]:
y_test_a[anomalies]
y_test_a[real_anomaly]

array([[-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.]])

In [146]:
anomalies.shape

(4, 10)